In [1]:
import numpy as np
import random
from datetime import *
import time
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [19]:
#Cargo csv
muertes = sqlContext.read.csv('info-muertes.csv', header=True).rdd.cache()

pobreza = sqlContext.read.csv('pobreza2.csv', header=True).rdd.cache()

In [47]:
#Item A

#mapeo como (state,povrty_rate)
pobreza_estado = pobreza.map(lambda x: (x[0],float(x[2])))
pobreza_estado = pobreza_estado.reduceByKey(lambda x,y: x+y)

#Ahora joineo con el rdd de pobreza, pero antes lo mapeo a (state,(city,poverty))
rdd1 = pobreza.map(lambda x: (x[0],(x[1],x[2])))

#me queda algo de la forma (state,(poverty_state,(city,poverty_city)))
pobreza_estado_ciudad = pobreza_estado.join(rdd1).cache()
#Calculo la diferencia
pobreza_estado_ciudad = pobreza_estado_ciudad.map(lambda x: (x[1][1][0], abs(float(x[1][1][1])-x[1][0])))
#Me quedo con las 10 con mayor diferencia
pobreza_estado_ciudad.takeOrdered(10,lambda x: -x[1])

                   

In [46]:
#Item B
#mapeo el rdd de muertes por (state,race) y me quedo solo con los de raza negra
rdd2 = muertes.filter(lambda x: x[2] =='black').map(lambda x: (x[4],x[2])) 
#joineo este rdd con el de pobreza_estado que contiene (state,poverty_rate_state) y redondeo
#los ratios de pobreza, quedando como (poverty_rate_round,1) #1 para sumar
rdd3 = rdd2.join(pobreza_estado).map(lambda x: (round(x[1][1]),1))
#Reduzco por clave y sumo
rdd3 = rdd3.reduceByKey(lambda x,y: x+y)
#aca habria que devolver el resultado